In [ ]:
# Spiel Vier Gewinnt

In [ ]:
import canvas_helpers #für optische Darstellungen
from ipycanvas import Canvas

import math # für math.ceil, wird benötigt um die Zahl zu runden

from IPython.display import display, clear_output # wird für das löschen des Spielfeldes benötigt

# Einmalige Definitionsgrössen
breite = hoehe = 300  # Breite vom Canvas, Erlaubter Wertebereich ohne Probleme: 200 bis 350
felderxy = 7  # Fixe Zahl (Spielfeld ist 7 Felder breit), Spielfeld ist 6 hoch, jedoch noch eine Zeile für Zeilenbeschriftung
feldgroessexy = breite / felderxy # berechnung des einzelnen Feldes in x
Kreisradius = feldgroessexy/3.5 # berechnung des Kreisradius in abhänigigkeit den Feldgrössen
Text = breite / 26 # Text wird grösse, wenn die Feldgrösse grösser wird
Zeilen = math.ceil(breite / 23) #anzahl maximaler Zeilen berechnen für log_canvas

class VierGewinnt:
    def __init__(self, rows=felderxy - 1, cols=felderxy, color='gelb'):
        # Initialisierung: Festlegung der grössen im board aus den Definitionsgrössen am Anfang
        # Zudem definition von Anfangszuständen, die sich während dem Spiel verändern
        self.rows = rows
        self.cols = cols
        self.board = [[' ' for _ in range(cols)] for _ in range(rows)]
        self.player = color # Anfangsspieler ist color = 'gelb'
        self.Frage = ' ' # Definiert die Abfrage der Spalte
        self.log = [] # Speicher für die Logdaten
        self.Eingabe = ' '
        self.Darstellung()

    def Darstellung(self): # Grundsätzliche Spielfelddarstellung und Logdatenfeld
        global canvas, log_canvas # => canvas als global definieren
        
        # definiton canvas
        canvas = Canvas(width=breite, height=hoehe,
                        layout={'border': '2px solid blue'})

        # Leinwand mit blauem Rechteck einfassen
        canvas.fill_style = 'blue'
        canvas.fill_rect(0, 0, canvas.width, canvas.height)

        # Zeichne Senkrechte linien zur unterteilung
        for i in range(1, felderxy):
            x = i * feldgroessexy
            canvas.stroke_lines([(x, 0), (x, hoehe)])

        # Zeichne Waagerechte linien zur unterteilung
        for i in range(1, felderxy):
            y = i * feldgroessexy
            canvas.stroke_lines([(0, y), (breite, y)])

        # Spaltenbeschriftung
        Zahlen = list(range(1, felderxy + 1))  # Speichert die Zahlen von 1 bis 7
        canvas.text_baseline = 'alphabetic'
        canvas.font = font = '{}px serif'.format(feldgroessexy)
        for i, text_align in enumerate(Zahlen):
            canvas.fill_style = 'black'
            canvas.fill_text(text_align, (feldgroessexy / 4) + (i * feldgroessexy), (feldgroessexy / 5 * 4))

        # Aussenrahmen
        canvas.stroke_style = 'black'
        canvas.stroke_rect(0, 0, canvas.width, canvas.height)
        
        # Darstellung der Kreise
        for row in range(self.rows):
            for col in range(self.cols):
                if self.board[row][col] == 'gelb':
                    canvas.fill_style = 'yellow'
                elif self.board[row][col] == 'rot':
                    canvas.fill_style = 'red'
                else:
                    canvas.fill_style = 'white'  # Leerfelder als weiß darstellen

                x = col * feldgroessexy + feldgroessexy / 2
                y = hoehe - ((self.rows - row - 1) * feldgroessexy + feldgroessexy / 2)  # Vertikale Umkehrung
                canvas.fill_circle(x, y, Kreisradius)

        # Log Canvas unter dem Spielfeldy
        log_canvas = Canvas(width=breite, height=hoehe, layout={'border': '2px solid green'})
        log_canvas.fill_style = 'white'
        log_canvas.fill_rect(0, 0, log_canvas.width, log_canvas.height)
        log_canvas.stroke_rect(0, 0, log_canvas.width, log_canvas.height)

        display(canvas, log_canvas)

    def gewonnen(self):# Funktion zum Überprüfen, ob ein Spieler gewonnen hat
        # Horizontal
        for row in range(self.rows):
            for col in range(self.cols - 3):
                if (self.board[row][col] == self.player and
                        self.board[row][col + 1] == self.player and
                        self.board[row][col + 2] == self.player and
                        self.board[row][col + 3] == self.player):
                    return True
        # Vertikal
        for row in range(self.rows - 3):
            for col in range(self.cols):
                if (self.board[row][col] == self.player and
                        self.board[row + 1][col] == self.player and
                        self.board[row + 2][col] == self.player and
                        self.board[row + 3][col] == self.player):
                    return True
        # Diagonal (rechts)
        for row in range(self.rows - 3):
            for col in range(self.cols - 3):
                if (self.board[row][col] == self.player and
                        self.board[row + 1][col + 1] == self.player and
                        self.board[row + 2][col + 2] == self.player and
                        self.board[row + 3][col + 3] == self.player):
                    return True
        # Diagonal (links)
        for row in range(3, self.rows):
            for col in range(self.cols - 3):
                if (self.board[row][col] == self.player and
                        self.board[row - 1][col + 1] == self.player and
                        self.board[row - 2][col + 2] == self.player and
                        self.board[row - 3][col + 3] == self.player):
                    return True
        # Rückgabe False, wenn kein Spieler gewonnen hat
        return False

    def Text_ist_Zahl(self): # Log-Eintrag hinzufügen
        self.log.append(f'Spieler {self.player}: Spalte {self.Eingabe + 1}')
        self.Text_schreiben()

    def Text_ist_falsch(self): # Text ist falsch oder Spielfeld voll oder ein Spieler hat gewonnen
        self.Text_schreiben()
     
    def Text_schreiben(self):
        # Wenn die Anzahl der Einträge im Log 'Zeilen' erreicht, von vorne beginnen
        if len(self.log) > Zeilen:
            self.log = self.log[-Zeilen:]  # Behalte nur die letzten 'Zeilen' Einträge
            
        # schreiben des Text
        log_canvas.clear()
        log_canvas.text_align = 'left'
        log_canvas.text_baseline = 'middle'
        log_canvas.font = '{}px serif'.format(Text) # Textgrösse in Abhängigkeit von Feldgrösse
        log_canvas.fill_style = 'black'
        for i, entry in enumerate(self.log):
            log_canvas.fill_text(entry, 10, 20 + i * 20)
    
    def Spielerwechsel(self): # Spielerwechsel jenachdem wer aktuell aktiv ist
        if self.player == 'rot':
            self.player = 'gelb'
        else:
            self.player = 'rot'

    def drop_piece(self, Eingabe): # Spielstein fallen lassen // Rückgabe false, falls spalte voll ist
        for row in range(self.rows - 1, -1, -1):
            if self.board[row][Eingabe] == ' ':
                self.board[row][Eingabe] = self.player
                return True
        return False

    def neues_Spiel(self): # Abfrage für neues Spiel 
        self.Frage = input("Wollt ihr noch eine Runde spielen? (ja/nein)")
        if self.Frage == "ja" or self.Frage == "nein": #kontrolle ob eingabe ja oder nein ist
            if self.Frage == "ja": # eingabe ja: relevante Spieldaten zurücksetzen
                clear_output()
                rows = felderxy - 1
                cols = felderxy
                self.board = [[' ' for _ in range(cols)] for _ in range(rows)]
                self.log = []  # Spielprotokoll zurücksetzen
                self.Darstellung()

            if self.Frage == "nein": # eingabe nein: Spielfeld löschen und spiel beenden
                clear_output(wait=True)
                print("Spiel beendet")
        if self.Frage != "ja" and self.Frage != "nein":
            print("Bitte mit ja oder nein Antworten.")
            self.neues_Spiel() # sorgt für endlosschleife, falls eingabe nicht ja oder nein

    def Abfrage(self): # Hauptschleife
        while True:
            # Spaltenabfrage
            self.Eingabe = input(f"Spieler {self.player}, wähle eine Spalte: ")
            if not self.Eingabe.isdigit(): # self.Eingabe ist Zahl?
                self.log.append('Die Eingabe muss eine Zahl sein. Bitte eine Zahl eingeben.')
                self.Text_ist_falsch()
                continue

            self.Eingabe = int(self.Eingabe) # self.Eingabe zu int ändern
            self.Eingabe -= 1 # Spalten sind 0 bis 6, Darstellung ist jedoch 1 bis 7

            if self.Eingabe < 0 or self.Eingabe >= self.cols: # Zahl ausserhalb der möglichen spalten
                self.log.append("Ungültige Eingabe. Bitte eine Spalte 1 bis 7 wählen.")
                self.Text_ist_falsch()                
                continue
                
            if not self.drop_piece(self.Eingabe): # Spielstein fallen lassen, falls spalte voll => neue eingabe nötig
                self.log.append("Diese Spalte ist voll. Bitte wählen Sie eine andere Spalte")
                self.Text_ist_falsch()                
                continue
                
            clear_output() # Löschen von allen anzeigen
                 
            self.Darstellung() # neuzeichnen anzeige
            self.Text_ist_Zahl() # text in log_canvas schreiben
            
            if self.gewonnen(): # kontrolle, ob ein spieler gewonnen hat
                self.log.append(f"Spieler {self.player} gewinnt. Gratulation!")
                self.Text_ist_falsch()
                self.neues_Spiel()
                
            if all(self.board[row][col] != ' ' for row in range(self.rows) for col in range(self.cols)): # Kontrolle, ob "Spielbrett" voll ist
                self.log.append("Es sind alle Spalten voll. Es gibt keinen Gewinner.")
                self.Text_ist_falsch()
                self.neues_Spiel()
                
            if self.Frage == "nein":# verarbeitung von Rückmeldung self.neues_Spiel. Hier break falls 'nein' um while zu beenden
                break

            self.Spielerwechsel() # Spielrunde vorbei, Spielerwechsel

In [ ]:
game = VierGewinnt()
game.Abfrage()